In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go


COMMON_LAYOUT = dict(
    width=1000,
    height=600,
    title_x=0.5,
    plot_bgcolor="#111111",
    paper_bgcolor="#111111",
    font=dict(color="white"),
    xaxis=dict(showgrid=False, zeroline=False, tickfont=dict(color="white")),
    # Removed yaxis from here
    legend=dict(
        font=dict(color="white"),
        orientation="h",
        x=0.5,
        xanchor="center",
        y=1.05,
    )
)


def create_choropleth_map(df):
    df = df.copy()
    fig = px.choropleth(
        df,
        locations="Country",
        locationmode="country names",
        color="Defense_Share_GDP",
        hover_name="Country",
        hover_data={"Defense_Share_GDP": ':.2f'},
        animation_frame="Year",
        title="🗺️ Global Defense Spending as % of GDP Over Time",
        template="plotly_dark",
        color_continuous_scale="Plasma"
    )

    fig.update_layout(
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='natural earth',
            bgcolor="#111111"
        ),
        coloraxis_colorbar=dict(
            title=dict(text="Defense Share of GDP (%)", font=dict(color="white")),
            tickfont=dict(color="white")
        ),
        **COMMON_LAYOUT
    )

    return fig


def create_defense_gdp_indexed_trend(df):
    df = df.dropna(subset=["Defense_USD", "GDP"]).copy()
    df["Year"] = df["Year"].astype(int)
    countries = sorted(df["Country"].unique())
    data_dict = {}

    for country in countries:
        df_c = df[df["Country"] == country].sort_values("Year")
        base_def = df_c["Defense_USD"].iloc[0]
        base_gdp = df_c["GDP"].iloc[0]

        df_c["Defense_Indexed"] = (df_c["Defense_USD"] / base_def) * 100
        df_c["GDP_Indexed"] = (df_c["GDP"] / base_gdp) * 100
        df_c["Defense_Hover"] = df_c.apply(lambda r: f"Year: {r['Year']}<br>Defense: {r['Defense_USD']:.2f}M<br>Indexed: {r['Defense_Indexed']:.2f}", axis=1)
        df_c["GDP_Hover"] = df_c.apply(lambda r: f"Year: {r['Year']}<br>GDP: {r['GDP']:.2f}M<br>Indexed: {r['GDP_Indexed']:.2f}", axis=1)

        data_dict[country] = {
            "year": df_c["Year"],
            "defense_indexed": df_c["Defense_Indexed"],
            "gdp_indexed": df_c["GDP_Indexed"],
            "defense_hover": df_c["Defense_Hover"],
            "gdp_hover": df_c["GDP_Hover"],
        }

    init_country = countries[0]
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=data_dict[init_country]["year"],
        y=data_dict[init_country]["defense_indexed"],
        text=data_dict[init_country]["defense_hover"],
        hoverinfo="text",
        mode="lines+markers",
        name="Defense Spending (Base 100)"
    ))
    fig.add_trace(go.Scatter(
        x=data_dict[init_country]["year"],
        y=data_dict[init_country]["gdp_indexed"],
        text=data_dict[init_country]["gdp_hover"],
        hoverinfo="text",
        mode="lines+markers",
        name="GDP (Base 100)"
    ))

    fig.update_layout(
        title=dict(
            text=f"📈 Defense Spending & GDP Indexed Trend: {init_country}",
            font=dict(size=18, color="white"),
            x=0.4
        ),
        updatemenus=[
            dict(
                type="dropdown",
                direction="down",
                showactive=True,
                x=1,
                y=1.15,
                xanchor="right",
                font=dict(color="white"),
                bgcolor="#333",
                buttons=[
                    dict(
                        label=country,
                        method="update",
                        args=[
                            {
                                "x": [data_dict[country]["year"]] * 2,
                                "y": [data_dict[country]["defense_indexed"], data_dict[country]["gdp_indexed"]],
                                "text": [data_dict[country]["defense_hover"], data_dict[country]["gdp_hover"]],
                            },
                            {
                                "title": f"📈 Defense Spending & GDP Indexed Trend: {country}"
                            }
                        ]
                    )
                    for country in countries
                ]
            )
        ],
        **COMMON_LAYOUT
    )

    return fig



def create_defense_vs_gdp_scatter_excluding_usa_china(df):
    df = df[
        (~df["Country"].isin(["United States", "China"]))
        & df["GDP"].notna()
        & df["Defense_USD"].notna()
    ]

    fig = px.scatter(
        df,
        x="Defense_USD",
        y="GDP",
        size="Defense_USD",
        color="Continent",
        hover_name="Country",
        text="Country",
        animation_frame="Year",
        animation_group="Country",
        size_max=60,
        title="📊 Defense Spending vs GDP Over Time (Excluding USA & China)",
        template="plotly_dark",
    )

    fig.update_traces(
        textposition="middle center",
        textfont=dict(size=8, color="black"),
        marker=dict(line=dict(width=0.5, color="black")),
    )

    layout_updates = COMMON_LAYOUT.copy()
    layout_updates["xaxis"] = dict(
        title="Defense Spending (in USD millions)",
        tickformat=",",
        range=[
            df["Defense_USD"].min() * 0.9,
            df["Defense_USD"].max() * 1.1,
        ],
        showgrid=False,
        zeroline=False,
        tickfont=dict(color="white"),
    )
    layout_updates["yaxis"] = dict(
        title="GDP (in USD millions)",
        tickformat=",",
        range=[
            df["GDP"].min() * 0.9,
            df["GDP"].max() * 1.3,
        ],
        showgrid=False,
        zeroline=False,
        tickfont=dict(color="white"),
    )

    fig.update_layout(**layout_updates)

    return fig


def create_defense_spending_over_time(df):
    df_time = df.groupby(["Year", "Continent"]).Defense_USD.sum().reset_index()
    fig = px.line(
        df_time,
        x="Year",
        y="Defense_USD",
        color="Continent",
        markers=True,
        title="📈 Defense Spending Over Time by Continent",
        template="plotly_dark"
    )
    fig.update_layout(
        hovermode="x unified",
        **COMMON_LAYOUT
    )
    return fig


def create_country_defense_bar_animation(df):
    df = df.copy()
    df["Year"] = df["Year"].astype(str)

    bloomberg_colors = [
        "#9467bd", "#393b79", "#7b4173", "#a55194", "#d6616b", "#3182bd"
    ]

    top_15_per_year = (
        df.groupby("Year", group_keys=False)
        .apply(lambda x: x.nlargest(15, "Defense_USD"))
        .copy()
    )

    top_15_per_year["Rank"] = (
        top_15_per_year.groupby("Year")["Defense_USD"]
        .rank(method="first", ascending=False)
    )
    top_15_per_year.sort_values(by=["Year", "Rank"], inplace=True)

    fig = px.bar(
        top_15_per_year,
        x="Defense_USD",
        y="Country",
        orientation="h",
        animation_frame="Year",
        color="Country",
        hover_name="Country",
        range_x=[0, top_15_per_year["Defense_USD"].max() * 1.1],
        template="plotly_dark",
        color_discrete_sequence=bloomberg_colors,
        labels={"Defense_USD": "Defense Spending (USD Millions)"},
    )

    fig.update_layout(
        title=dict(
            text="💰 Animated Country Defense Spending Over Time (Top 15 per Year)",
            font=dict(size=18, color="white"),
            x=0.5,
            xanchor="center"
        ),
        yaxis=dict(
            title="",
            categoryorder="total ascending"
        ),
        showlegend=False,
        **COMMON_LAYOUT
    )

    return fig


def create_country_defense_trend(df):
    selected_countries = st.multiselect(
        "Select Countries to View Trends",
        options=sorted(df["Country"].unique()),
        default=["India", "Germany", "China"]
    )

    if not selected_countries:
        st.warning("⚠️ Please select at least one country.")
        return None

    df_filtered = df[df["Country"].isin(selected_countries)].copy()
    fig = px.line(
        df_filtered,
        x="Year",
        y="Defense_USD",
        color="Country",
        markers=True,
        title="📈 Defense Spending Over Time by Country",
        template="plotly_dark"
    )
    fig.update_layout(
        hovermode="x unified",
        **COMMON_LAYOUT
    )
    return fig

# Add any other defense spending visualizations you want here,
# using the same pattern for consistent style.


In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from src.utils import country_to_iso3, get_country_coords_from_csv as get_country_coords

@st.cache_data
def load_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["Year"] = df["Year"].astype(int)
    return df

def create_choropleth_map(df: pd.DataFrame) -> go.Figure:
    df = df.copy()
    df["ISO3"] = df["Country"].apply(country_to_iso3)

    fig = px.choropleth(
        df,
        locations="Country",
        locationmode="country names",
        color="Defense_Share_GDP",
        hover_name="Country",
        hover_data={"Defense_Share_GDP": ':.2f'},
        animation_frame="Year",
        title="🗺️ Global Defense Spending as % of GDP Over Time",
        template="plotly_dark",
        color_continuous_scale="Plasma"
    )

    # Add country labels for the first year
    earliest_year = df['Year'].min()
    df_labels = df[df['Year'] == earliest_year].copy()

    country_coords = get_country_coords(tuple(df_labels["Country"]))
    df_labels["lat"] = df_labels["Country"].map(lambda x: country_coords.get(x, (None, None))[0])
    df_labels["lon"] = df_labels["Country"].map(lambda x: country_coords.get(x, (None, None))[1])
    df_labels = df_labels.dropna(subset=["lat", "lon"])

    #fig.add_trace(go.Scattergeo(
        #lon=df_labels["lon"],
        #lat=df_labels["lat"],
        #text=df_labels["Country"],
        #mode="text",
        #showlegend=False,
        #textfont=dict(color="white", size=8),
    #))

    # Layout and styling
    fig.update_layout(
        width=1000,
        height=600,
        margin=dict(l=0, r=0, t=50, b=0),
        title=dict(
            font=dict(size=20, color="white"),
            x=0.5,
            xanchor="center"
        ),
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='natural earth',
            bgcolor="#111111"
        ),
        coloraxis_colorbar=dict(
            title=dict(text="Defense Share of GDP (%)", font=dict(color="white")),
            tickfont=dict(color="white")
        ),
        paper_bgcolor="#111111",
        plot_bgcolor="#111111"
    )

    return fig


# 20️⃣ Indexed Defense Spending and GDP Over Time
def create_defense_gdp_indexed_trend(df):
    df = df.dropna(subset=["Defense_USD", "GDP"]).copy()
    df["Year"] = df["Year"].astype(int)
    countries = sorted(df["Country"].unique())
    data_dict = {}

    for country in countries:
        df_c = df[df["Country"] == country].sort_values("Year")
        base_def = df_c["Defense_USD"].iloc[0]
        base_gdp = df_c["GDP"].iloc[0]

        df_c["Defense_Indexed"] = (df_c["Defense_USD"] / base_def) * 100
        df_c["GDP_Indexed"] = (df_c["GDP"] / base_gdp) * 100
        df_c["Defense_Hover"] = df_c.apply(lambda r: f"Year: {r['Year']}<br>Defense: {r['Defense_USD']:.2f}M<br>Indexed: {r['Defense_Indexed']:.2f}", axis=1)
        df_c["GDP_Hover"] = df_c.apply(lambda r: f"Year: {r['Year']}<br>GDP: {r['GDP']:.2f}M<br>Indexed: {r['GDP_Indexed']:.2f}", axis=1)

        data_dict[country] = {
            "year": df_c["Year"],
            "defense_indexed": df_c["Defense_Indexed"],
            "gdp_indexed": df_c["GDP_Indexed"],
            "defense_hover": df_c["Defense_Hover"],
            "gdp_hover": df_c["GDP_Hover"],
        }

    init_country = countries[0]
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=data_dict[init_country]["year"],
        y=data_dict[init_country]["defense_indexed"],
        text=data_dict[init_country]["defense_hover"],
        hoverinfo="text",
        mode="lines+markers",
        name="Defense Spending (Base 100)"
    ))
    fig.add_trace(go.Scatter(
        x=data_dict[init_country]["year"],
        y=data_dict[init_country]["gdp_indexed"],
        text=data_dict[init_country]["gdp_hover"],
        hoverinfo="text",
        mode="lines+markers",
        name="GDP (Base 100)"
    ))

    fig.update_layout(
        template="plotly_dark",
        title=dict(
            text=f"📈 Defense Spending & GDP Indexed Trend: {init_country}",
            font=dict(size=18, color="white"),
            x=0.4
        ),
        xaxis_title="Year",
        yaxis_title="Indexed Value (Base 100)",
        plot_bgcolor="#111111",
        paper_bgcolor="#111111",
        legend=dict(
            font=dict(color="white"),
            orientation="h",
            x=0.5,
            xanchor="center"
        ),
        height=600,
        updatemenus=[
            dict(
                type="dropdown",
                direction="down",
                showactive=True,
                x=1,
                y=1.15,
                xanchor="right",
                font=dict(color="white"),
                bgcolor="#333",
                buttons=[
                    dict(
                        label=country,
                        method="update",
                        args=[
                            {
                                "x": [data_dict[country]["year"]] * 2,
                                "y": [data_dict[country]["defense_indexed"], data_dict[country]["gdp_indexed"]],
                                "text": [data_dict[country]["defense_hover"], data_dict[country]["gdp_hover"]],
                            },
                            {
                                "title": f"📈 Defense Spending & GDP Indexed Trend: {country}"
                            }
                        ]
                    )
                    for country in countries
                ]
            )
        ]
    )

    return fig


def create_defense_vs_gdp_scatter_excluding_usa_china(df):
    df = df[
        ~df["Country"].isin(["United States", "China"])
        & df["GDP"].notna()
        & df["Defense_USD"].notna()
    ]

    fig = px.scatter(
        df,
        x="Defense_USD",
        y="GDP",
        size="Defense_USD",
        color="Continent",
        hover_name="Country",
        text="Country",
        animation_frame="Year",
        animation_group="Country",
        size_max=60,
        title="Defense Spending vs GDP Over Time (Excluding USA & China)",
        template="plotly_dark",
    )

    fig.update_traces(
        textposition="middle center",
        textfont=dict(size=8, color="black"),
        marker=dict(line=dict(width=0.5, color="black")),
    )

    fig.update_layout(
        title=dict(
            text="📊 Defense Spending vs GDP Over Time (Excluding USA & China)",
            font=dict(size=18, color="white"),
            x=0.5,
        ),
        xaxis=dict(
            title="Defense Spending (in USD millions)",
            tickfont_color="white",
            tickformat=",",
            range=[
                df["Defense_USD"].min() * 0.9,
                df["Defense_USD"].max() * 1.1,
            ],
            showgrid=False,
            zeroline=False,
        ),
        yaxis=dict(
            title="GDP (in USD millions)",
            tickfont_color="white",
            tickformat=",",
            range=[
                df["GDP"].min() * 0.9,
                df["GDP"].max() * 1.3,
            ],
            showgrid=False,
            zeroline=False,
        ),
        plot_bgcolor="#111111",
        paper_bgcolor="#111111",
        legend=dict(
            font_color="gray",
            orientation="h",
            x=0.5,
            xanchor="center",
            y=1.05,
        ),
        height=600,
    )
    return fig

# 5 Line Chart — Total Defense Spending Over Time by Continent
def create_defense_spending_over_time(df):
    df_time = df.groupby(["Year", "Continent"]).Defense_USD.sum().reset_index()
    fig = px.line(
        df_time,
        x="Year",
        y="Defense_USD",
        color="Continent",
        markers=True,
        title="Defense Spending Over Time by Continent"
    )
    fig.update_layout(
        title_x=0.5,
        plot_bgcolor="black",
        paper_bgcolor="black",
        font=dict(color="white"),
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        hovermode="x unified"
    )
    return fig

import streamlit as st
import plotly.express as px

def create_country_defense_bar_animation(df):
    st.markdown("### 🏆 Animated Top 15 Countries by Defense Spending Over Time")

    # Ensure 'Year' is string for animation frame
    df["Year"] = df["Year"].astype(str)

    # Bloomberg-style color palette
    bloomberg_colors = [
        "#9467bd",  # purple
        "#393b79",  # dark blue
        "#7b4173",  # dark purple
        "#a55194",  # medium purple
        "#d6616b",  # light red
        "#3182bd",  # medium blue
    ]

    # Get top 15 countries per year dynamically
    top_15_per_year = (
        df.groupby("Year", group_keys=False)
        .apply(lambda x: x.nlargest(15, "Defense_USD"))
        .copy()
    )

    # Rank for sorting bars correctly in animation
    top_15_per_year["Rank"] = (
        top_15_per_year.groupby("Year")["Defense_USD"]
        .rank(method="first", ascending=False)
    )
    top_15_per_year.sort_values(by=["Year", "Rank"], inplace=True)

    # Plot
    fig = px.bar(
        top_15_per_year,
        x="Defense_USD",
        y="Country",
        orientation="h",
        animation_frame="Year",
        color="Country",
        hover_name="Country",
        range_x=[0, top_15_per_year["Defense_USD"].max() * 1.1],
        template="plotly_dark",
        color_discrete_sequence=bloomberg_colors,
        labels={"Defense_USD": "Defense Spending (USD Millions)"},
    )

    # Layout and styling
    fig.update_layout(
        title=dict(
            text="💰 Animated Country Defense Spending Over Time (Top 15 per Year)",
            font=dict(size=18, color="white"),
            x=0.5,
            xanchor="center"
        ),
        xaxis=dict(
            title="Defense Spending (in USD millions)",
            tickfont=dict(color="white"),
            showgrid=False,
            zeroline=False,
            tickformat=","
        ),
        yaxis=dict(
            title="",
            tickfont=dict(color="white"),
            categoryorder="total ascending",
            showgrid=False,
            zeroline=False
        ),
        plot_bgcolor="#111111",
        paper_bgcolor="#111111",
        font=dict(color="white"),
        showlegend=False,
        height=650,
    )

    return fig



# 7 Line Chart — Country Defense Spending Trends
def create_country_defense_trend(df):
    selected_countries = st.multiselect(
        "Select Countries to View Trends",
        options=sorted(df["Country"].unique()),
        default=["India", "Germany", "China"]
    )

    if not selected_countries:
        st.warning("⚠️ Please select at least one country.")
        return None

    df_filtered = df[df["Country"].isin(selected_countries)].copy()
    fig = px.line(
        df_filtered,
        x="Year",
        y="Defense_USD",
        color="Country",
        markers=True,
        title="Defense Spending Over Time by Country"
    )
    fig.update_layout(
        title_x=0.5,
        hovermode="x unified",
        plot_bgcolor="black",
        paper_bgcolor="black",
        font=dict(color="white"),
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False)
    )
    return fig

